In [ ]:
# { 'default_profile': [False], 'profile_background_color': ['1A1B1F'], 'description': [''], 'location': ['Duluth/Atlanta, GA'], 'id_str': ['443226440'], 'contributors_enabled': [False], 'followers_count': [16], 'friends_count': [127], 'name': ['Tony Schlitt'], 'utc_offset': [-14400], 'profile_sidebar_border_color': ['181A1E'], 'profile_background_image_url_https': ['https://abs.twimg.com/images/themes/theme9/bg.gif'], 'id': [443226440], 'favourites_count': [226], 'profile_use_background_image': [True], 'status': { 'in_reply_to_status_id': {}, 'id_str': ['783437675328135173'], 'retweet_count': [1], 'is_quote_status': [False], 'retweeted': [False], 'in_reply_to_user_id': [442246443], 'in_reply_to_screen_name': ['brianhoyt24'], 'truncated': [False], 'source': ['Twitter for iPhone'], 'favorite_count': [1], 'id': [7.83437675328135e+17], 'geo': {}, 'contributors': {}, 'lang': ['en'], 'favorited': [False], 'created_at': ['Tue Oct 04 22:44:32 +0000 2016'], 'text': ['@brianhoyt24 @FrontOfficeLos I did the same as Hoyt 😢, but you have to play a "Woo! off" between Hoyt and Dimino\'s woos! 😂😂😂'], 'in_reply_to_user_id_str': ['442246443'], 'coordinates': {}, 'in_reply_to_status_id_str': {}, 'entities': { 'hashtags': [], 'symbols': [], 'user_mentions': [{ 'indices': [ [0], [12] ], 'screen_name': ['brianhoyt24'], 'name': ['Happy Hoyt'], 'id': [442246443], 'id_str': ['442246443'] }, { 'indices': [ [13], [28] ], 'screen_name': ['FrontOfficeLos'], 'name': ['Los Medina'], 'id': [487944445], 'id_str': ['487944445'] }], 'urls': [] }, 'place': { 'bounding_box': { 'type': ['Polygon'], 'coordinates': [ [ [ [-85.6052], [30.3556] ], [ [-80.7426], [30.3556] ], [ [-80.7426], [35.0008] ], [ [-85.6052], [35.0008] ] ] ] }, 'place_type': ['admin'], 'attributes': {}, 'country': ['United States'], 'contained_within': [], 'id': ['7142eb97ae21e839'], 'country_code': ['US'], 'url': ['https://api.twitter.com/1.1/geo/id/7142eb97ae21e839.json'], 'name': ['Georgia'], 'full_name': ['Georgia, USA'] } }, 'profile_image_url': ['http://pbs.twimg.com/profile_images/1883818109/Al3rB_CCIAAfSdq_normal.jpg'], 'verified': [False], 'is_translator': [False], 'created_at': ['Wed Dec 21 23:35:27 +0000 2011'], 'profile_background_image_url': ['http://abs.twimg.com/images/themes/theme9/bg.gif'], 'url': {}, 'profile_link_color': ['2FC2EF'], 'time_zone': ['Eastern Time (US & Canada)'], 'is_translation_enabled': [False], 'statuses_count': [600], 'has_extended_profile': [False], 'profile_sidebar_fill_color': ['252429'], 'notifications': [False], 'default_profile_image': [False], 'screen_name': ['printpirate'], 'geo_enabled': [True], 'lang': ['en'], 'listed_count': [0], 'following': [False], 'profile_image_url_https': ['https://pbs.twimg.com/profile_images/1883818109/Al3rB_CCIAAfSdq_normal.jpg'], 'entities': { 'description': { 'urls': [] } }, 'follow_request_sent': [False], 'protected': [False], 'profile_background_tile': [False], 'profile_text_color': ['666666'] }


In [1]:
"# extract the given column info, produce a dataframe\n",
"#Note :1, the field in user json can be dict or list. this function not handle dict now.\n",
"#      the NULL, will give a complete graph, which expand all the dict as well.       \n",
"#      2, some lines of the downloaded json items are not valid, use try except to get rid of them\n",

import re
import json
import pandas as pd 
import csv
   

def json2dataframe(json_file, cols = None, verbose = False):
    """
    json_file, a filepath to a json file containing multiple json objects
    cols: extract the selected keys, or extract all the keys, and return a dataframe
    verbose: to output the nuber of invalid json lines
    rtype: panda DataFrame
    """
    if cols:
        data = {}
        n_invalids = 0
        for c in cols:
            data[c] = []
        with open (json_file) as f1:
            for line in f1:
                try:
                    line = json.loads(line) #case 1, not valid json, pass
                except:
                    n_invalids += 1    
                    continue
                if 'id_str' not in line:   #case 2, valid.json stype, no id_str, pass
                    continue
                for c in cols:             
                    if c not in line or isinstance(line[c], dict) :
                        data[c].append('NA_VALUE')
                    elif  line[c] == []: #include empty list
                        data[c].extend('NA_VALUE') 
                    else:
                        aa = str(line[c][0])
                        data[c].append( re.sub("\\s+"," ",re.sub("[\n\r\t]+", " ", aa)) )  
        data_df = pd.DataFrame.from_dict(data)
    else:
        json_list = []
        n_invalids = 0
        with open (json_file) as f1:
            for line in f1:
                try:
                    line = json.loads(line)
                    json_list.append(line)
                except:
                    n_invalids += 1
                    continue           
        data_df = pd.DataFrame(json_list) 
    if verbose: print("number of invalid lines removed: " + str(n_invalids))
    return data_df

In [2]:
#example on small json file
cols = ['id_str', 'screen_name', 'name','created_at', 'description', 
        'location', 'time_zone',  'favourites_count','followers_count',  'friends_count',
        'lang', 'listed_count' ,'statuses_count', 'geo_enabled', 'protected' ,'verified'] 
path = "../../data/followers_info/followers10.json"
cols1 = ['id_str', 'screen_name','description','followers_count']
toys_df1 = json2dataframe(path, cols1)
print(len(toys_df1.columns))

cols = ['id_str', 'screen_name', 'name','created_at', 'description', 
        'location', 'time_zone',  'favourites_count','followers_count',  'friends_count',
        'lang', 'listed_count' ,'statuses_count', 'geo_enabled', 'protected' ,'verified'] 
toys_df2 = json2dataframe(path, cols)
print(len(toys_df2.columns))

toys_df3 = json2dataframe(path, verbose = True)
print(len(toys_df3.columns))

4
16
number of invalid lines removed: 0
42


In [4]:
#toys_df1.to_csv("followers10.csv", header = False)
toys_df2

,created_at,description,favourites_count,followers_count,friends_count,geo_enabled,id_str,lang,listed_count,location,name,protected,screen_name,statuses_count,time_zone,verified
0,Sun Jul 03 21:09:26 +0000 2016,,0,10,52,False,749711671782961152,en,0,,mungai simon ngigi,False,mungaisimonngi2,0,NA_VALUE,False
1,Wed Aug 31 01:01:00 +0000 2016,,0,3,19,False,770788443609829376,ru,0,,Анастасия Антонова,False,dyachkovalyuci6,0,NA_VALUE,False
2,Fri Sep 16 14:24:48 +0000 2016,,0,1,25,False,776788934177292288,en,0,,Donna Tyrone,False,tyrone_donna,1,NA_VALUE,False
3,Thu Jul 21 15:45:15 +0000 2016,,0,0,19,False,756153070594039808,en,0,,Chappy Politics,True,ChappyPolitics,0,NA_VALUE,False
4,Tue May 29 20:37:42 +0000 2012,I am very much friendly and jovial and have a ...,19,15,335,True,594099777,en,0,India,Sarat Banerjee,False,banerjeesarat,31,Kolkata,False
5,Sat Jul 18 10:43:30 +0000 2015,ACHS Freshman,209,24,91,False,3381564803,en,1,"Lawrenceburg, KY",Ethan Smith,False,EthanSm95233995,0,NA_VALUE,False
6,Tue Jul 12 13:06:30 +0000 2016,Love taking pictures❤ Single Fun Love shopping...,0,8,102,False,752851627720241152,en,0,"Loganville, GA",Troyann Campbell,False,Thickbitvht,2,NA_VALUE,False
7,Mon May 16 17:53:18 +0000 2016,I am an Oklahoma State University grad. I have...,8,2,40,False,732267694855053313,en,0,"Oklahoma, USA",Cassnloganmom,False,AbrahamLincln82,0,NA_VALUE,False
8,Wed Sep 14 13:30:02 +0000 2016,#Election2016 Art Arts & Culture Business & Fi...,0,0,36,False,776050374914535425,en,0,"Nashville, TN",Qisen Song,False,nealolaen,0,NA_VALUE,False
9,Sun Jul 03 14:27:43 +0000 2016,,0,19,122,False,749610578121854976,vi,0,青森 五所川原市,Hoàng Thị Yến,False,Hg1Yen,0,NA_VALUE,False


In [5]:
#feb 17, 2017
# Mar22,
# MAr29
# April 13
#cols = ['id_str', 'screen_name','description','followers_count']
cols = ['id_str', 'screen_name', 'name','created_at', 'description', 
        'location', 'time_zone',  'favourites_count','followers_count',  'friends_count',
        'lang', 'listed_count' ,'statuses_count', 'geo_enabled', 'protected' ,'verified'] 
#decription, need to do text cleaning before write into .csv
for i in range(0,16):
    if i < 10:
        inputpath = "../../data/followers_info/jsons/trump0"+str(i)+".json"
        outputpath = "../../data/followers_info/csvs/trump0"+str(i)+".csv"
    elif i < 100:
        inputpath = "../../data/followers_info/jsons/trump"+str(i)+".json"
        outputpath = "../../data/followers_info/csvs/trump"+str(i)+".csv"
    trump_df = json2dataframe(inputpath, cols, verbose = True)  
    print ("i=" +  str(i) +" is done , sample size: " + str(len(trump_df)))
    trump_df.to_csv(outputpath)

number of invalid lines removed: 2
i=0 is done , sample size: 803920
number of invalid lines removed: 1
i=1 is done , sample size: 949999
number of invalid lines removed: 0
i=2 is done , sample size: 949999
number of invalid lines removed: 3
i=3 is done , sample size: 949996
number of invalid lines removed: 3
i=4 is done , sample size: 949997
number of invalid lines removed: 3
i=5 is done , sample size: 850202
number of invalid lines removed: 0
i=6 is done , sample size: 933486
number of invalid lines removed: 1
i=7 is done , sample size: 800000
number of invalid lines removed: 0
i=8 is done , sample size: 800000
number of invalid lines removed: 1
i=9 is done , sample size: 800000
number of invalid lines removed: 1
i=10 is done , sample size: 800000
number of invalid lines removed: 1
i=11 is done , sample size: 800000
number of invalid lines removed: 1
i=12 is done , sample size: 800000
number of invalid lines removed: 0
i=13 is done , sample size: 800000
number of invalid lines remove

In [ ]:
'''
number of invalid lines removed: 2
i=0 is done , sample size: 803920
number of invalid lines removed: 1
i=1 is done , sample size: 949999
number of invalid lines removed: 0
i=2 is done , sample size: 949999
number of invalid lines removed: 3
i=3 is done , sample size: 949996
number of invalid lines removed: 3
i=4 is done , sample size: 949997
number of invalid lines removed: 3
i=5 is done , sample size: 850202
number of invalid lines removed: 0
i=6 is done , sample size: 933486
number of invalid lines removed: 1
i=7 is done , sample size: 800000
number of invalid lines removed: 0
i=8 is done , sample size: 800000
number of invalid lines removed: 1
i=9 is done , sample size: 800000
number of invalid lines removed: 1
i=10 is done , sample size: 800000
number of invalid lines removed: 1
i=11 is done , sample size: 800000
number of invalid lines removed: 1
i=12 is done , sample size: 800000
number of invalid lines removed: 0
i=13 is done , sample size: 800000
number of invalid lines removed: 0
i=14 is done , sample size: 800000
number of invalid lines removed: 0
i=15 is done , sample size: 91887

'''


In [2]:
# process all the friends.json, in three graphs
# paril 17, 2017

#cols = ['id_str', 'screen_name','description','followers_count']
cols = ['id_str', 'screen_name', 'name','created_at', 'description', 
        'location', 'time_zone',  'favourites_count','followers_count',  'friends_count',
        'lang', 'listed_count' ,'statuses_count', 'geo_enabled', 'protected' ,'verified'] 


inputpath = "../../data/friends_info/data_friends/friends_1.json"
outputpath = "../../data/friends_info/data_friends/friends_1_info.csv"
df = json2dataframe(inputpath, cols, verbose = True)  
print ("sample size: " + str(len(df)))
df.to_csv(outputpath)


inputpath = "../../data/friends_info/data_friends/friends_2.json"
outputpath = "../../data/friends_info/data_friends/friends_2_info.csv"
df = json2dataframe(inputpath, cols, verbose = True)  
print ("sample size: " + str(len(df)))
df.to_csv(outputpath)
          
inputpath = "../../data/friends_info/data_friends/friends_3.json"
outputpath = "../../data/friends_info/data_friends/friends_3_info.csv"
df = json2dataframe(inputpath, cols, verbose = True)  
print ("sample size: " + str(len(df)))
df.to_csv(outputpath)         

number of invalid lines removed: 68
sample size: 1944633
number of invalid lines removed: 138
sample size: 4100181
number of invalid lines removed: 117
sample size: 3160939


In [ ]:
'''
number of invalid lines removed: 68
sample size: 1944633
number of invalid lines removed: 138
sample size: 4100181
number of invalid lines removed: 117
sample size: 3160939
'''